本实验使用Qwen2.5-1.5B模型 测试
仅用于学习不用于生成最终模型 - 因此数据量也较低

### 1. 数据处理 - Tokenizer
#### 1.1 加载数据 - Dataset

In [1]:
import pandas as pd
from datasets import Dataset
train = pd.read_csv("data/train.csv")
train_ds = Dataset.from_pandas(train)
train_ds[:5]

{'text': ['现头昏口苦',
  '目的观察复方丁香开胃贴外敷神阙穴治疗慢性心功能不全伴功能性消化不良的临床疗效',
  '舒肝和胃消痞汤；功能性消化不良',
  '患者３ａ前咯血，被诊断为肺结核，住院４０余天时出现腹痛，经治疗好转，但时有发作，坚持服抗痨药３ａ后，因腹痛基本缓解，肺结核治愈而停药',
  '治疗组采用复方蜥蜴散不同微粒组合剂（密点麻蜥、炙黄芪、焦乌梅、炒白芍、三七、半枝莲等）治疗'],
 'label': ["{'口苦': '临床表现'}",
  "{'复方丁香开胃贴': '中医治疗', '心功能不全伴功能性消化不良': '西医诊断'}",
  "{'功能性消化不良': '西医诊断'}",
  "{'咯血': '临床表现', '肺结核': '西医诊断'}",
  "{'复方蜥蜴散': '方剂', '密点麻蜥': '中药', '炙黄芪': '中药', '焦乌梅': '中药', '炒白芍': '中药', '三七': '中药', '半枝莲': '中药'}"]}

#### 1.2 Tokenization
+ 加载tokenizer 
+ 定义process function：prompt方程
+ 处理dataset为 [input_id, attention_mask,labels]

In [3]:
from modelscope import AutoTokenizer
model_dir ="/root/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B/snapshots/d149729398750b98c0af14eb82c78cfe92750796"  # 定义本地路径
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False, trust_remote_code=True) # 加载tokenizer

In [4]:
# 定义process function:
def process_func(example):
    MAX_LENGTH = 300
    instruction = """你是一个文本实体识别领域的医学专家，你需要从给定的句子中提取中医诊断,中药,中医治疗, 方剂, 西医治疗, 西医诊断 '其他治疗'等. 
    注意: 1. 如果找到任何实体, 输出必须是严格的json字符串. 如 {'口苦': '临床表现','肺结核': '西医诊断'}。
    2.找不到任何实体时, 输出"没有找到任何实体".
    3.避免输出重复的内容"""
    instructions_messages= [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{example['text']}"}
    ]
    response_messages=[
            {"role": "assistant", "content":f"{example['label']}"},
        ]
    # Chat-template + tokenizer 处理
    instructions_chatTamplate = tokenizer.apply_chat_template(instructions_messages, tokenize=True, add_generation_prompt=False,return_dict=True)
    total_chatTamplate = tokenizer.apply_chat_template(instructions_messages + response_messages, tokenize=True, padding=True,truncation=True,max_length=MAX_LENGTH,add_generation_prompt=False,return_dict=True)

    # 凑labels的内容
    instruction_len = len(instructions_chatTamplate['input_ids'])
    labels = [-100] * instruction_len + total_chatTamplate['input_ids'][instruction_len:]
    input_ids = total_chatTamplate['input_ids']
    attention_mask = total_chatTamplate['attention_mask']
    
    # 限制最大长度做截断处理
    if len(input_ids) > MAX_LENGTH:
        print(len(input_ids))
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}  

In [5]:
train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names,num_proc=4) 
# 删除column_names 保证简洁性
# 多线程处理数据 加速数据处理速度

Map (num_proc=4):   0%|          | 0/5259 [00:00<?, ? examples/s]

### 2. 训练模型
#### 2.1 加载模型

In [7]:
from transformers import AutoModelForCausalLM
import torch
model = AutoModelForCausalLM.from_pretrained(model_dir,torch_dtype=torch.float16)
model.enable_input_require_grads()  # 开启梯度检查点时，要执行该方法 

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

#### 2.2 定义Lora参数

In [8]:
# 导包
import torch
from peft import LoraConfig, TaskType, get_peft_model
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

# Lora微调参数
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, # 实现方式为生成式，因此选CAUSAL_LM 不选 TOKEN_CLASSIFICATION
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, 
    r=8, 
    lora_alpha=32,  
    lora_dropout=0.1,  
)
model = get_peft_model(model, config)

In [9]:
model.print_trainable_parameters() # 检查模型可训练参数大小

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


### 2.3 定义Trainer参数 并开始训练

In [10]:
# 训练参数
args = TrainingArguments(
    output_dir="./output/Qwen2-NER-Doctor", # 模型输出地址
    per_device_train_batch_size=2, # 每张显卡上 的 batch_size 一个GPU 就是4
    gradient_accumulation_steps=4, # 累计多少步更新一次参数，通常会比per_device_train_batch_size大
    logging_steps=10,
    num_train_epochs=2, # 训练的轮次
    save_steps=50,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
    fp16=False, # fp16需要N卡
    bf16=True, # mps 仅支持高精度训练 
)

In [11]:
#设置 Trainer 开始训练
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding="max_length",max_length=300)
)

trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,1.459300
20,0.835800
30,0.873800
40,0.727100
50,0.669900
60,0.724400
70,0.787500
80,0.905300
90,0.828500
100,0.740900


TrainOutput(global_step=1316, training_loss=0.7282299556993062, metrics={'train_runtime': 2009.971, 'train_samples_per_second': 5.233, 'train_steps_per_second': 0.655, 'total_flos': 1.342459418904576e+17, 'train_loss': 0.7282299556993062, 'epoch': 2.0})

In [13]:
# 保存LoRA权重（仅几MB~几十MB，无需保存完整大模型）
trainer.model.save_pretrained("./lora_weights_qwen2.5_7B")